In [1]:
import pandas as pd

In [2]:
output_folder = '/mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Output_files/'
input_folder = '/mbshome/nvelthuijs/Cofactors/20200402_Transcriptionregulationlist/Datafiles/'
file1 = output_folder + '10_Transcription_regulatorlist_baitset1_with_scores.csv'
file2 = output_folder + '11_Transcription_regulatorlist_baitset2_with_scores.csv'
file3 = output_folder + '12_Transcription_regulatorlist_baitset3_with_scores.csv'
filter_terms_file = input_folder + 'filter_terms.txt'
gtf_terms_file = input_folder + 'gtf_terms.txt'

In [3]:
with open(filter_terms_file) as filter_terms:
    filter_terms = [line.strip().split('\t') for line in filter_terms]
    
with open(gtf_terms_file) as gtf_terms:
    gtf_terms = set([line.strip().split('\t')[0] for line in gtf_terms])

for file, baitset in zip([file1, file2, file3], [1, 2, 3]):
    with open(file) as full_list:
        df = pd.read_csv(full_list)
        
    new_cols = {'ribosome' : [],
                'spliceosome' : [],
                'proteasome' : []
                }

    for index, row in df.iterrows():
        filter_values = {'ribosome' : 0,
                         'spliceosome' : 0,
                         'proteasome' : 0
                        }
        
        given_terms = {'CC' : row['CC terms'],
                       'BP' : row['BP terms'],
                       'MF' : row['MF terms']
                      }
        
        for filter_term in filter_terms:
            term = filter_term[0]
            namespace = filter_term[1]
            proteincomplex = filter_term[2]
            
            terms = given_terms[namespace]
            if isinstance(terms, float):
                continue
            terms = set(terms[2:-2].replace("'", "").replace(' ', '').split(','))
            if isinstance(terms, set) and term in terms:
                filter_values[proteincomplex] = 1
                
        for key, value in filter_values.items():
            new_cols[key].append(value)
            
    for key, value in new_cols.items():
        df['Blacklist {}'.format(key)] = value
    
    df['Blacklist any'] = df.loc[:, ['Blacklist ribosome', 'Blacklist spliceosome', 'Blacklist proteasome']].any(axis = 1).astype(int)

    gtf_col = []
    for index, row in df.iterrows():
        cc_terms = row['MF terms']
        if isinstance(cc_terms, float):
            gtf_col.append(0)
            continue
        cc_terms = set(cc_terms[2:-2].replace("'", "").replace(' ', '').split(','))
        if cc_terms.intersection(gtf_terms):
            gtf_col.append(1)
        else:
            gtf_col.append(0)
    df['Is_GTF'] = gtf_col
        
    output_filename = '{}{}_Transcription_regulatorlist_baitset{}_with_scores_with_blacklist.csv'.format(output_folder, str(baitset + 13), str(baitset))
    df.to_csv(output_filename, index = False)

for x in ['any', 'ribosome', 'spliceosome', 'proteasome']:
    print(df['Blacklist {}'.format(x)].value_counts(), '\n')
df

0    3396
1     407
Name: Blacklist any, dtype: int64 

0    3623
1     180
Name: Blacklist ribosome, dtype: int64 

0    3618
1     185
Name: Blacklist spliceosome, dtype: int64 

0    3759
1      44
Name: Blacklist proteasome, dtype: int64 



,Gene Symbol,UniProt ID,ENSG ID,ComplexID_huMAP,Bait_huMAP,ComplexID_CORUM,ComplexName_CORUM,PMID_CORUM,Bait_CORUM,ComplexName_nvs,...,Total GO score,Total GO score penalty,GO score,Mine score,Final score,Blacklist ribosome,Blacklist spliceosome,Blacklist proteasome,Blacklist any,Is_GTF
0,AATF,Q9NY61,ENSG00000275700,Drew1337;Drew1809;Drew3735,RIOX1;ZNF629;DHX30;ZBTB24;ZBTB11;MYBBP1A;PURA;...,NaN,NaN,NaN,NaN,NaN,...,3.072727,-5,-0.642424,3,2.357576,0,0,0,0,0
1,ABL1,P00519,ENSG00000097007,Drew2259,ABL1,6592;6076;2811,c-Abl-cortactin-nmMLCK complex;BRCA1-cABL comp...,20861316;12024016;23740246,BRCA1;ABL1,NaN,...,4.171591,0,1.390530,5,6.390530,0,0,0,0,0
2,ABRAXAS1,Q6UWZ7,ENSG00000163322,Drew1841,UIMC1,2786,BRCA1 A complex,17525340,BRCA1;UIMC1,NaN,...,1.747727,-8,-2.084091,2,-0.084091,0,0,0,0,0
3,ACIN1,Q9UKV3,ENSG00000100813,Drew2802;Drew4295;Drew1090,SRSF2;SPEN;RBM39;THRAP3;BCLAF1;CDC73,351;760,Spliceosome;Apoptosis- and splicing-associated...,12226669;12665594,DDX5;SRSF2;DHX9;SPEN;BCAS2;SF1;SAP18;DDX17;SNW...,NaN,...,1.747727,-10,-2.750758,2,-0.750758,0,0,0,0,0
4,ACOT1,Q86TX2,ENSG00000184227,Drew956,TFAP2A,NaN,NaN,NaN,NaN,NaN,...,0.000000,-9,-3.000000,2,-1.000000,0,0,0,0,0
5,ACOT8,O14734,ENSG00000101473,Drew1995,PML,NaN,NaN,NaN,NaN,NaN,...,0.000000,-12,-4.000000,2,-2.000000,0,0,0,0,0
6,ACTB,P60709,ENSG00000075624,Drew2569;Drew233;Drew4201;Drew1704,CORO2A;GSN;ACTB;LRRFIP1;BASP1,725;5615;1729;7298;5606;6984;1166;5736;5607;56...,CTGF/Hcs24-actin(beta/gamma) complex;Emerin ar...,14729568;15834686;12470643;17620012;15502823;1...,PBRM1;HDAC1;PARP1;PTMA;GATAD2B;SMARCC2;DPY30;N...,BAF;PBAF;NuA4,...,4.361364,0,1.453788,5,6.453788,0,0,0,0,0
7,ACTG1,P63261,ENSG00000184009,Drew2569;Drew780,GSN;ACTN4;ACTN1;ACTB;CFL1;HMGB2;BASP1;ZBTB20,189;2254,BAF complex;CTGF/Hcs24-actin(beta/gamma) complex,11078522;12470643,SMARCC1;ARID1A;ACTB;SMARCC2;ACTL6B;SMARCD1;SMA...,NaN,...,0.902273,-10,-3.032576,2,-1.032576,0,0,0,0,0
8,ACTL6A,O96019,ENSG00000136518,Drew4093;Drew2673,PBRM1;BRD9;BRD7;SMARCC2;SMARCA4;SMARCD2;BICRAL...,1251;807;304;564;238;565;1166;1175;798;1230;55...,BRM-SIN3A-HDAC complex;WINAC complex;BRG1-SIN3...,9710619;8895581;11073988;11780067;10882073;984...,PBRM1;BRD7;HDAC1;GATAD2B;H2AZ1;SMARCC2;MBD3;CC...,BAF;PBAF;NuA4;SRCAP;INO80;GBAF,...,7.886364,0,2.628788,5,7.628788,0,0,0,0,0
9,ACTL6B,O94805,ENSG00000077080,Drew4093;Drew2673,PBRM1;BRD9;BRD7;SMARCC2;SMARCA4;SMARCD2;BICRAL...,189,BAF complex,11078522,SMARCC1;ARID1A;ACTB;SMARCC2;ACTL6B;SMARCD1;SMA...,BAF;PBAF,...,5.960227,0,1.986742,3,4.986742,0,0,0,0,0
